In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')

### Imports

In [2]:
import os
import pandas as pd
import tqdm
import re
import torch
import json
# import torch_xla
# import torch_xla.core.xla_model as xm
from torch.utils.data import Dataset, TensorDataset, DataLoader, SequentialSampler, RandomSampler
from torch.nn.utils.rnn import pad_sequence
# from keras.preprocessing.sequence import pad_sequences
import pickle
import numpy as np
import tensorflow as tf
from preprocess import *
from vocabulary import *
from model import *
from model_functions import *

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\swaga\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\swaga\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\swaga\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\swaga\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize 
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from sklearn.preprocessing import LabelBinarizer
import seaborn as sns
from string import punctuation
import re
from gensim.models import KeyedVectors
import numpy as np

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\swaga\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\swaga\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\swaga\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Pre Processing Dataset

In [4]:
train_df = pd.read_csv('../../../evaluation/snli/snli_/snli_1.0_train.csv',  encoding='utf-8')
val_df   = pd.read_csv('../../../evaluation/snli/snli_/snli_1.0_dev.csv',  encoding='utf-8')
# train_df = pd.read_csv('../datasets/snli/snli_/snli_1.0_train.csv')
# val_df   = pd.read_csv('../datasets/snli/snli_/snli_1.0_dev.csv')
train_df, val_df = preprocess_text(train_df, val_df)
print(len(train_df))
print(len(val_df))

549361
9842


In [5]:
# len(train_df['sentence2'][8])

In [6]:
# rw = pd.read_csv('../datasets/Transformer_generated_summaries.csv', encoding='utf-8')
# rw = pd.read_csv('ins.csv', encoding='utf-8')
# rw = pd.read_csv('test_sets/ins.csv', encoding='ansi')
rw = pd.read_csv('test_sets/based_ins.csv', encoding='utf-8')
# rw = pd.read_csv('../datasets/test_data/Transformer_generated_summaries.csv', encoding='utf-8')

rw.drop(['original_summary'], axis = 1, inplace=True)
rw.rename(columns = {'text':'sentence1'}, inplace = True)
# rw.rename(columns = {'Predicted_summary':'sentence2'}, inplace = True)
rw.rename(columns = {'predicted_summary':'sentence2'}, inplace = True)

# print(rw)
gold = [-1] * 100
rw.insert(loc=0, column='gold_label', value=gold)
rw = rw[['gold_label', 'sentence1', 'sentence2']]
# rw = rw[['sentence1', 'sentence2']]
rw = rw.dropna()
rw.head()

,gold_label,sentence1,sentence2
0,-1,saudi stock exchange appointed sarah al ceo in...,saudi arabia sign 1st ever female woman
1,-1,hollywood actor morgan freeman said would like...,new zealand unveils new bike with
2,-1,uk india business council chairperson said 60 ...,india to increase taxes to boost growth in india
3,-1,indian railways wednesday increased lower bert...,railways to increase in india
4,-1,two pilots arrested saturday glasgow scotland ...,two arrested for using migrant plane in japan


In [7]:
# rw = pd.read_csv('../datasets/Transformer_generated_summaries.csv', encoding='utf-8')
# rw = pd.read_csv('ins.csv', encoding='utf-8')
# rw_ = pd.read_csv('test_sets/ins.csv', encoding='ansi')
rw_ = pd.read_csv('test_sets/based_ins.csv', encoding='utf-8')
# rw = pd.read_csv('../datasets/test_data/Transformer_generated_summaries.csv', encoding='utf-8')

rw_.drop(['text'], axis = 1, inplace=True)
rw_.rename(columns = {'original_summary':'sentence1'}, inplace = True)
# rw_.rename(columns = {'Predicted_summary':'sentence2'}, inplace = True)
rw_.rename(columns = {'predicted_summary':'sentence2'}, inplace = True)

gold = [-1] * 100
rw_.insert(loc=0, column='gold_label', value=gold)
rw_ = rw_[['gold_label', 'sentence1', 'sentence2']]
# rw = rw[['sentence1', 'sentence2']]
rw_ = rw_.dropna()
rw_.head()

,gold_label,sentence1,sentence2
0,-1,saudi stock exchange appoints its first female...,saudi arabia sign 1st ever female woman
1,-1,what makes fireworks explode,new zealand unveils new bike with
2,-1,60 british firms to increase investment in india,india to increase taxes to boost growth in india
3,-1,railways increase senior citizen women quota,railways to increase in india
4,-1,pilots arrested in scotland for drinking befor...,two arrested for using migrant plane in japan


In [8]:
rw_.head()

,gold_label,sentence1,sentence2
0,-1,saudi stock exchange appoints its first female...,saudi arabia sign 1st ever female woman
1,-1,what makes fireworks explode,new zealand unveils new bike with
2,-1,60 british firms to increase investment in india,india to increase taxes to boost growth in india
3,-1,railways increase senior citizen women quota,railways to increase in india
4,-1,pilots arrested in scotland for drinking befor...,two arrested for using migrant plane in japan


In [9]:
rw['sentence1'][0]

'saudi stock exchange appointed sarah al ceo investment bank ncb capital first female head first woman chair major government financial institution kingdom chair arab world 39 largest stock exchange time preparing offer shares public '

In [10]:
rw_['sentence2'][0]

' saudi arabia sign 1st ever female woman'

In [11]:
len(rw['sentence2'][8])

42

In [12]:
train_df['sentence1'] = train_df['sentence1'].astype(str).apply(lambda text: clean_text(text))
train_df['sentence2'] = train_df['sentence2'].astype(str).apply(lambda text: clean_text(text))
val_df['sentence1'] = val_df['sentence1'].astype(str).apply(lambda text: clean_text(text))
val_df['sentence2'] = val_df['sentence2'].astype(str).apply(lambda text: clean_text(text))

In [13]:
rw['sentence1'] = rw['sentence1'].astype(str).apply(lambda text: clean_text(text))
rw['sentence2'] = rw['sentence2'].astype(str).apply(lambda text: clean_text(text))

In [14]:
rw_['sentence1'] = rw_['sentence1'].astype(str).apply(lambda text: clean_text(text))
rw_['sentence2'] = rw_['sentence2'].astype(str).apply(lambda text: clean_text(text))

In [15]:
train_df = train_df[(train_df['sentence1'].str.split().str.len() > 0) & (train_df['sentence2'].str.split().str.len() > 0)]
val_df = val_df[(val_df['sentence1'].str.split().str.len() > 0) & (val_df['sentence2'].str.split().str.len() > 0)]
print(train_df[(train_df['sentence1'].str.split().str.len() == 0) | (train_df['sentence2'].str.split().str.len() == 0)])
print(val_df[(val_df['sentence1'].str.split().str.len() == 0) | (val_df['sentence2'].str.split().str.len() == 0)])

Empty DataFrame
Columns: [gold_label, sentence1, sentence2]
Index: []
Empty DataFrame
Columns: [gold_label, sentence1, sentence2]
Index: []


In [16]:
# train_df.head()

In [17]:
# val_df.head()

In [18]:
# train_val_df = pd.concat([train_df, val_df])

In [19]:
# sentence_pairs, _ = pair_generator(train_val_df)
rw_sentence_pairs, __ = pair_generator(rw)
rw_sentence_pairs_, __ = pair_generator(rw_)
train_sentence_pairs, train_sentence_labels = pair_generator(train_df)
val_sentence_pairs, val_sentence_labels = pair_generator(val_df)

labels = set(train_sentence_labels)
print(labels)

# tag2idx = {word: i for i, word in enumerate(labels)}
# print(tag2idx)
tag2idx = {'entailment': 0, 'neutral': 1, 'contradiction': 2}
print(tag2idx)

train_labels = [tag2idx[t] for t in train_sentence_labels]
val_labels = [tag2idx[t] for t in val_sentence_labels]

{'entailment', 'contradiction', 'neutral'}
{'entailment': 0, 'neutral': 1, 'contradiction': 2}


In [20]:
train_sentence_pairs[0]

('a person on a horse jumps over a broken down airplane',
 'a person is training his horse for a competition')

In [21]:
# vocab = Vocabulary()

# for data in [rw_sentence_pairs]:
#   for sen in data:
#     premise    = sen[0]
#     hypothesis = sen[1]
#     vocab.addSentence(premise)
#     vocab.addSentence(hypothesis)
    
# for data in [rw_sentence_pairs_]:
#   for sen in data:
#     premise    = sen[0]
#     hypothesis = sen[1]
#     vocab.addSentence(premise)
#     vocab.addSentence(hypothesis)

# for data in [sentence_pairs]:
#   for sentence_pair in data:
#     premise    = sentence_pair[0]
#     hypothesis = sentence_pair[1]
#     vocab.addSentence(premise)
#     vocab.addSentence(hypothesis)

# with open('saved_vocabs/voc.class', 'wb') as vocab_file:
#     pickle.dump(vocab, vocab_file)
# print("Vocab size:", len(vocab.word2index))

In [22]:
with open('saved_vocabs/based_voc.class', 'rb') as vocab_file_r:
    vocab = pickle.load(vocab_file_r)
print("Vocab size:", len(vocab.word2index))

Vocab size: 33862


In [24]:
vocab.word2index['thoes']

2616

In [25]:
index2word = {}
for wrd, idx in vocab.word2index.items():
    # print(wrd, idx)
    index2word[idx] = wrd

## Model

In [26]:
EPOCHS = 15
BATCH_SIZE = 32
EMBEDDING_SIZE = 300
VOCAB_SIZE = len(vocab.word2index)
TARGET_SIZE = len(tag2idx)
HIDDEN_SIZE = 128
LEARNING_RATE = 0.005
STACKED_LAYERS = 2
EMBEDDING_PATH = '../../../../embeddings/google_news/GoogleNews-vectors-negative300.bin'
GLOVE_EMBEDDING = '../../../../embeddings/glove/glove.6B.300d.txt'
# EMBEDDING_PATH = '../embeddings/google_news/GoogleNews-vectors-negative300.bin'
# GLOVE_EMBEDDING = '../embeddings/glove/glove.6B.300d.txt'

initiate_model_vocab(vocab, tag2idx)

In [27]:
train_data = DataSetLoader(get_pair_indices(vocab, train_sentence_pairs), train_labels)
val_data   = DataSetLoader(get_pair_indices(vocab, val_sentence_pairs), val_labels)

In [28]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size = BATCH_SIZE, collate_fn=lambda x:x)
val_loader   = torch.utils.data.DataLoader(val_data, batch_size = BATCH_SIZE, collate_fn=lambda x:x)

print(len(train_loader), len(val_loader))

17168 308


In [29]:
embeddings_index = load_embeddings(GLOVE_EMBEDDING)

In [30]:
weights = 1 * np.random.randn(VOCAB_SIZE + 1, EMBEDDING_SIZE)
embedded_count = 0
for word, lang_word_index in vocab.word2index.items():
  if embeddings_index.get(word) is not None:
    weights[lang_word_index] = embeddings_index.get(word)
    embedded_count += 1

print("Embedded count:", embedded_count)
del embeddings_index
weights.shape

Embedded count: 27677


(33863, 300)

In [31]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [32]:
lstm_model = LSTM(VOCAB_SIZE, HIDDEN_SIZE, TARGET_SIZE, STACKED_LAYERS, weights, True)
lstm_model.to(device)
print(lstm_model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm_model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)

LSTM(
  (embedding): Embedding(33863, 300)
  (lstm): LSTM(300, 128, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (FC_concat1): Linear(in_features=512, out_features=128, bias=True)
  (FC_concat2): Linear(in_features=128, out_features=64, bias=True)
  (FC_concat3): Linear(in_features=64, out_features=32, bias=True)
  (output): Linear(in_features=32, out_features=3, bias=True)
  (out): Sequential(
    (0): Linear(in_features=512, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Linear(in_features=64, out_features=32, bias=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): Linear(in_features=32, out_features=3, bias=True)
  )
)


In [33]:
train(lstm_model, train_loader, val_loader, criterion, optimizer)

  7%|█████▎                                                                         | 1/15 [24:53<5:48:26, 1493.35s/it]

00:24:53.34
Epoch 1: train_loss: 0.8261 train_acc: 0.6387 | val_loss: 0.7226 val_acc: 0.6976


 13%|██████████▌                                                                    | 2/15 [48:46<5:15:51, 1457.82s/it]

00:23:52.92
Epoch 2: train_loss: 0.7546 train_acc: 0.6829 | val_loss: 0.6987 val_acc: 0.7136


 20%|███████████████▍                                                             | 3/15 [1:09:05<4:29:45, 1348.80s/it]

...

00:12:11.46
Epoch 13: train_loss: 0.7090 train_acc: 0.7054 | val_loss: 0.6605 val_acc: 0.7338


 93%|█████████████████████████████████████████████████████████████████████████▋     | 14/15 [3:38:07<12:38, 758.51s/it]

00:12:13.21
Epoch 14: train_loss: 0.7086 train_acc: 0.7064 | val_loss: 0.6746 val_acc: 0.7253


100%|███████████████████████████████████████████████████████████████████████████████| 15/15 [3:50:22<00:00, 921.52s/it]

00:12:14.91
Epoch 15: train_loss: 0.7119 train_acc: 0.7050 | val_loss: 0.6746 val_acc: 0.7247


In [34]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in lstm_model.state_dict():
    print(param_tensor, "\t", lstm_model.state_dict()[param_tensor].size())

Model's state_dict:
embedding.weight 	 torch.Size([33863, 300])
lstm.weight_ih_l0 	 torch.Size([512, 300])
lstm.weight_hh_l0 	 torch.Size([512, 128])
lstm.bias_ih_l0 	 torch.Size([512])
lstm.bias_hh_l0 	 torch.Size([512])
lstm.weight_ih_l0_reverse 	 torch.Size([512, 300])
lstm.weight_hh_l0_reverse 	 torch.Size([512, 128])
lstm.bias_ih_l0_reverse 	 torch.Size([512])
lstm.bias_hh_l0_reverse 	 torch.Size([512])
lstm.weight_ih_l1 	 torch.Size([512, 256])
lstm.weight_hh_l1 	 torch.Size([512, 128])
lstm.bias_ih_l1 	 torch.Size([512])
lstm.bias_hh_l1 	 torch.Size([512])
lstm.weight_ih_l1_reverse 	 torch.Size([512, 256])
lstm.weight_hh_l1_reverse 	 torch.Size([512, 128])
lstm.bias_ih_l1_reverse 	 torch.Size([512])
lstm.bias_hh_l1_reverse 	 torch.Size([512])
FC_concat1.weight 	 torch.Size([128, 512])
FC_concat1.bias 	 torch.Size([128])
FC_concat2.weight 	 torch.Size([64, 128])
FC_concat2.bias 	 torch.Size([64])
FC_concat3.weight 	 torch.Size([32, 64])
FC_concat3.bias 	 torch.Size([32])
output.w

In [35]:
# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

Optimizer's state_dict:
state 	 {0: {'step': 257520, 'exp_avg': tensor([[-1.4013e-45,  2.8026e-45, -0.0000e+00,  ...,  2.8026e-45,
          2.8026e-45,  1.4013e-45],
        [-5.6052e-45,  4.2039e-45,  4.2039e-45,  ..., -4.2039e-45,
          1.4013e-45,  5.6052e-45],
        [ 3.2836e-05, -5.1622e-05,  4.0932e-05,  ...,  1.9181e-05,
         -1.8662e-05, -9.7921e-06],
        ...,
        [-2.8026e-45, -1.4013e-45,  4.2039e-45,  ..., -0.0000e+00,
         -1.4013e-45, -0.0000e+00],
        [ 0.0000e+00,  4.2039e-45, -1.4013e-45,  ...,  1.4013e-45,
          2.8026e-45, -2.8026e-45],
        [ 1.4013e-45, -0.0000e+00, -2.8026e-45,  ...,  2.8026e-45,
         -0.0000e+00,  2.8026e-45]], device='cuda:0'), 'exp_avg_sq': tensor([[7.0065e-43, 7.0065e-43, 7.0065e-43,  ..., 7.0065e-43, 7.0065e-43,
         7.0065e-43],
        [2.4747e-13, 3.4147e-13, 3.5776e-13,  ..., 2.3316e-13, 5.5069e-13,
         4.2099e-13],
        [1.7348e-09, 4.5516e-08, 1.4363e-08,  ..., 2.1020e-08, 7.5986e-09,
   

In [36]:
torch.save(lstm_model, 'saved_models/based_model')

In [37]:
lstm_model = torch.load('saved_models/based_model')
lstm_model.eval()

LSTM(
  (embedding): Embedding(33863, 300)
  (lstm): LSTM(300, 128, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (FC_concat1): Linear(in_features=512, out_features=128, bias=True)
  (FC_concat2): Linear(in_features=128, out_features=64, bias=True)
  (FC_concat3): Linear(in_features=64, out_features=32, bias=True)
  (output): Linear(in_features=32, out_features=3, bias=True)
  (out): Sequential(
    (0): Linear(in_features=512, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Linear(in_features=64, out_features=32, bias=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): Linear(in_features=32, out_features=3, bias=True)
  )
)

In [ ]:
print(lstm_model.output)

In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(f'runs/SNLI/tensorboard_test')

In [ ]:
sentence_o = [''] * len(rw)
sentence_p = [''] * len(rw)
# sentence_o = ['']
# sentence_p = ['']

# sentence_o[0] = "rock music is very good for public"
# sentence_p[0] = "people dont like rock and roll"
# sentence_o = "uk india business council chairperson said 60 british firms likely increase investment india next years firms positive reforms fdi introduced last two years added india even important britain said"
# sentence_p = "60 british firms to increase investment in india"
# sentence_o = clean_text(sentence_o)
# sentence_p = clean_text(sentence_p)


# sen_p1 = [(sentence_o, sentence_p)]
# # sen_p2 = []
# for i in range(rw):
#   sen1 = sentence_o[i]
#   sen2 = sentence_p[i]
#   sen_p1.append((sen1, sen2))
# for i in range(rw_):
#   sen2 = sentence_o[i]
#   sen1 = sentence_p[i]
#   sen_p2.append((sen1, sen2))

# sen_p1

In [38]:
rrw_sentence_pairs_ = []
for i in range(len(rw_sentence_pairs_)):
    rrw_sentence_pairs_.append((rw_sentence_pairs_[i][1], rw_sentence_pairs_[i][0]))
print('rw[0]: ', rw_sentence_pairs_[0])
print('rrw[0]: ', rrw_sentence_pairs_[0])

rw[0]:  ('saudi stock exchange appoints its first female head', 'saudi arabia sign st ever female woman')
rrw[0]:  ('saudi arabia sign st ever female woman', 'saudi stock exchange appoints its first female head')


In [39]:
# rw_sentence_pairs, __ = pair_generator(rw)

In [40]:
id_pairs = get_pair_indices(vocab, rw_sentence_pairs)
id_pairs_1 = get_pair_indices(vocab, rw_sentence_pairs_)
id_pairs_2 = get_pair_indices(vocab, rrw_sentence_pairs_)
# id_pairs = get_pair_indices(vocab, sen_p1)
# id_pairs = get_pair_indices(vocab, sen_p2)
# id_pairs[0]

In [41]:
id_pairs

[([1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   12,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   16,
   22,
   23,
   24,
   2,
   3,
   25,
   26,
   27,
   28,
   29],
  [1, 30, 31, 32, 33, 13, 15]),
 ([34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   36,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   43,
   27,
   51,
   52,
   53,
   54,
   55,
   56,
   38],
  [57, 58, 59, 57, 60, 61]),
 ([62,
   43,
   63,
   64,
   65,
   38,
   66,
   67,
   68,
   69,
   8,
   43,
   70,
   71,
   67,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   71,
   49,
   43,
   79,
   80,
   81,
   82,
   38],
  [43, 83, 69, 84, 83, 85, 86, 87, 43]),
 ([88,
   89,
   90,
   91,
   92,
   93,
   94,
   95,
   96,
   97,
   98,
   99,
   100,
   13,
   101,
   102,
   71,
   103,
   104,
   105,
   106,
   107,
   108,
   94,
   109,
   110,
   111,
   112,
   113,
   114,
   115,
   116,
   117,
   118],
  [89, 83, 6

In [42]:
premise_seq    = [torch.tensor(seq[0]).long().to(device) for seq in id_pairs]
hypothesis_seq = [torch.tensor(seq[1]).long().to(device) for seq in id_pairs]

premise_len    = list(map(len, premise_seq))
hypothesis_len = list(map(len, hypothesis_seq))

batch = len(premise_seq)
temp = pad_sequence(premise_seq + hypothesis_seq, batch_first=True)
premise_seq    = temp[:batch, :]
hypothesis_seq = temp[batch:, :]

prediction = lstm_model([premise_seq, hypothesis_seq], premise_len, hypothesis_len)
# prediction = prediction[prediction!=prediction[0,3]]

prediction_list = prediction.to('cpu')
prediction_list = prediction_list.tolist()
print(len(prediction_list))

100


In [43]:
premise_seq    = [torch.tensor(seq[0]).long().to(device) for seq in id_pairs_1]
hypothesis_seq = [torch.tensor(seq[1]).long().to(device) for seq in id_pairs_1]

premise_len    = list(map(len, premise_seq))
hypothesis_len = list(map(len, hypothesis_seq))

batch = len(premise_seq)
temp = pad_sequence(premise_seq + hypothesis_seq, batch_first=True)
premise_seq    = temp[:batch, :]
hypothesis_seq = temp[batch:, :]

prediction_1 = lstm_model([premise_seq, hypothesis_seq], premise_len, hypothesis_len)
# prediction = prediction[prediction!=prediction[0,3]]

prediction_list = prediction_1.to('cpu')
prediction_list = prediction_list.tolist()
print(len(prediction_list))

100


In [44]:
premise_seq    = [torch.tensor(seq[0]).long().to(device) for seq in id_pairs_2]
hypothesis_seq = [torch.tensor(seq[1]).long().to(device) for seq in id_pairs_2]

premise_len    = list(map(len, premise_seq))
hypothesis_len = list(map(len, hypothesis_seq))

batch = len(premise_seq)
temp = pad_sequence(premise_seq + hypothesis_seq, batch_first=True)
premise_seq    = temp[:batch, :]
hypothesis_seq = temp[batch:, :]

prediction_2 = lstm_model([premise_seq, hypothesis_seq], premise_len, hypothesis_len)
# prediction = prediction[prediction!=prediction[0,3]]

prediction_list = prediction_2.to('cpu')
prediction_list = prediction_list.tolist()
print(len(prediction_list))

100


In [45]:
prediction

tensor([[ 3.1238e-01,  7.7762e-01, -6.3652e-01],
        [-9.2623e-01,  5.9577e-01,  6.2718e-01],
        [ 9.4998e-01,  9.6632e-01, -1.1888e+00],
        [-1.1257e-01,  1.1309e+00, -5.1142e-01],
        [ 9.6772e-01, -2.0223e-01, -8.8829e-01],
        [ 3.9615e-01,  1.8501e-01, -4.6335e-01],
        [ 1.0138e+00, -4.1150e-02, -9.8802e-01],
        [ 1.8298e-01,  9.4231e-02, -2.1156e-01],
        [ 6.4905e-01,  1.6825e-01, -7.0319e-01],
        [ 6.3601e-01, -3.7848e-03, -6.2575e-01],
        [ 2.5562e-01,  8.3978e-01, -6.2220e-01],
        [ 5.3703e-01,  2.4256e-01, -6.2310e-01],
        [ 6.3138e-02,  3.8222e-01, -2.9690e-01],
        [ 6.0854e-01, -1.8441e-01, -5.4036e-01],
        [-2.6121e-01, -6.0709e-01,  8.1202e-01],
        [ 5.7421e-01, -1.8597e-01, -5.0690e-01],
        [-5.9518e-01, -1.9367e-01,  7.9524e-01],
        [-2.5743e-01,  4.6726e-01, -3.1709e-02],
        [ 5.4734e-01,  1.5186e+00, -1.0996e+00],
        [ 2.6805e-01,  1.1327e+00, -7.6960e-01],
        [ 3.2408e-01

In [46]:
soft = torch.log_softmax(prediction, dim=1).argmax(dim=1)
soft.tolist()
soft_1 = torch.log_softmax(prediction_1, dim=1).argmax(dim=1)
soft_1.tolist()
soft_2 = torch.log_softmax(prediction_2, dim=1).argmax(dim=1)
soft_2.tolist()

[1,
 2,
 0,
 2,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 2,
 0,
 1,
 2,
 1,
 0,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 1,
 2,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 2,
 0,
 2,
 2,
 2,
 1,
 0,
 1,
 0,
 2,
 1,
 1,
 0,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 0,
 1,
 1,
 2,
 1,
 2,
 1,
 0,
 2,
 1,
 1]

In [47]:
print(soft, soft_1, soft_2)

tensor([1, 2, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 2, 0, 2, 1, 1, 1, 1, 0, 1, 0,
        1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 2, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0,
        0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 2, 2, 0, 1, 0, 0, 1, 0, 1, 0,
        0, 0, 0, 0, 2, 0, 2, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
        1, 0, 0, 1], device='cuda:0') tensor([0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 2, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1,
        1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 2, 1, 2, 1, 0, 2, 1, 1, 0, 1, 1, 1, 1, 0,
        2, 2, 2, 1, 1, 1, 2, 0, 1, 0, 0, 1, 1, 2, 2, 1, 0, 2, 1, 0, 1, 1, 1, 2,
        2, 0, 2, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1,
        1, 1, 2, 2], device='cuda:0') tensor([1, 2, 0, 2, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 0, 1, 2, 1,
        0, 1, 1, 1, 2, 2, 2, 1, 1, 2, 0, 1, 1, 1, 1, 1, 0, 2, 0, 2, 2, 2, 1, 0,
        1, 0, 2, 1, 1, 0, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 0, 1, 0, 1, 1, 1,
        1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 

In [51]:
ctr = 0
for i in range(len(soft)):
    if soft[i] == 0:
        print(rw_sentence_pairs[i])
        ctr = ctr + 1
print(ctr)

('two pilots arrested saturday glasgow scotland suspicion consuming alcohol prepared fly plane us police said two men fly passengers booked railways transport safety act two pilots reportedly removed flying duties pending investigation', 'two arrested for using migrant plane in japan')
('african american high school student franklin sought million damages awarded policemen unlawfully beat four years ago policemen entered franklin home without permission resembled another criminal description also held franklin custody resisting detention even realising mistake', 'france football team fined for mn')
('india first cosmetics brand lakm founded jrd tata subsidiary tata oil mills business tycoon born july requested prime minister jawaharlal nehru manufacture beauty products country brand named french opera lakm french form indian goddess lakshmi', 'india s first ac jaguar to reach tcs')
('year old artist uses melted create futuristic architectural paintings sculptures uses kite paper variou

In [49]:
ctr_1 = 0
for i in range(len(soft_1)):
    if soft_1[i] == 2:
        print(rw_sentence_pairs_[i])
        ctr_1 = ctr_1 + 1
print(ctr_1)

('panasonic launches two new headphones in india', 'panasonic launched for speeding')
('du from delhi this year', 'maharashtra extends year old households in')
('chinese village has sets of twins', 'chinese man becomes oldest monk')
('cricketers can be sent off for misbehaving under new rules', 'sl cricket team proposes pitch to tackle ultra league')
('italy bans trucks in cities after german terror attack', 'berlin truck attack suspect shot dead in brussels')
('poor affects rd humans report', 'global population may be shut down assocham')
('muslims should not waste their vote on sp mayawati', 'bsp bsp amit shah to move bsp bsp')
('world s tallest roller coaster opens in us', 'world s tallest indoor theme park is the world')
('ex manager appointed as kolkata coach', 'zidane s comeback to be real madrid')
('researchers identify drug target to stop dengue', 'hiv vaccine found in')
('sc to hear fresh plea to ban sikh jokes', 'sc refuses to stay sc to lift ban on jallikattu')
('bayern muni

In [50]:
ctr_2 = 0
for i in range(len(soft_2)):
    if soft_2[i] == 0:
        print(rrw_sentence_pairs_[i])
        ctr_2 = ctr_2 + 1
print(ctr_2)

('india to increase taxes to boost growth in india', 'british firms to increase investment in india')
('india s first ac jaguar to reach tcs', 'jrd tata founded india s first cosmetics brand')
('nitish kumar to get liquor to avoid crime', 'nda slams nitish govt over law and order issue')
('union budget session to be paid by cr', 'out of union ministers are crorepatis')
('maharashtra extends year old households in', 'du from delhi this year')
('startup lets users find d printed baby s glasses', 'uk startup lets users raise funds via selfies')
('bjp mp to join congress in arunachal', 'former bjp leader joins congress')
('raises k in seed funding', 'raises k in seed funding')
('global population may be shut down assocham', 'poor affects rd humans report')
('india s un envoy to un india s aggression', 'let jem should be treated as terror groups india at un')
('mobile mobile app will grow fold in fy bharti', 'crore phones in india in')
('refugees in france to be held in myanmar', 'cross bor

In [ ]:
# id_pairs = get_pair_indices(vocab, rrw_sentence_pairs)
# # id_pairs = get_pair_indices(vocab, sen_p2)
# id_pairs[0]

In [ ]:
# premise_seq    = [torch.tensor(seq[0]).long().to(device) for seq in id_pairs]
# hypothesis_seq = [torch.tensor(seq[1]).long().to(device) for seq in id_pairs]

# premise_len    = list(map(len, premise_seq))
# hypothesis_len = list(map(len, hypothesis_seq))

# batch = len(premise_seq)
# temp = pad_sequence(premise_seq + hypothesis_seq, batch_first=True)
# premise_seq    = temp[:batch, :]
# hypothesis_seq = temp[batch:, :]

# prediction = lstm_model([premise_seq, hypothesis_seq], premise_len, hypothesis_len)
# # prediction = prediction[prediction!=prediction[0,3]]

# prediction_list = prediction.to('cpu')
# prediction_list = prediction_list.tolist()
# print(len(prediction_list))

In [ ]:
# prediction

In [ ]:
# soft2 = torch.log_softmax(prediction, dim=1).argmax(dim=1)
# soft2.tolist()

In [ ]:
# ctr2 = 0
# for i in range(len(soft)):
#     if soft2[i] == 0:
#         print(rrw_sentence_pairs[i])
#         ctr2 = ctr2 + 1
# print(ctr2)

In [52]:
e_100 = 0   # perfectly entailed pairs
e_50 = 0    # entailment
n_100 = 0   # perfectly neutral pairs
n_50 = 0    # neutral
cc = 0      # contradicting pairs
for i in range(len(soft_1)):
    if soft_1[i] == soft_2[i] == 0:
        e_100 = e_100 + 1
    elif (soft_1[i] == 0 and soft_2[i] == 1) or (soft_1[i] == 1 and soft_2[i] == 0) or (soft_1[i] == 0 and soft_2[i] == 2) or (soft_1[i] == 2 and soft_2[i] == 0):
        e_50 = e_50 + 1
    elif soft_1[i] == soft_2[i] == 1:
        n_100 = n_100 + 1
    elif (soft_1[i] == 1 and soft_2[i] == 2) or (soft_1[i] == 2 and soft_2[i] == 1):
        n_50 = n_50 + 1
    else: 
        cc = cc + 1
print('e100: ', e_100)
print('e50: ', e_50)
print('n100: ', n_100)
print('n50: ', n_50)
print('c100: ', cc)

e100:  4
e50:  35
n100:  26
n50:  33
c100:  2


In [53]:
rw_sentence_pairs[0]

('saudi stock exchange appointed sarah al ceo investment bank ncb capital first female head first woman chair major government financial institution kingdom chair arab world largest stock exchange time preparing offer shares public',
 'saudi arabia sign st ever female woman')

In [54]:
prediction[0]

tensor([ 0.3124,  0.7776, -0.6365], device='cuda:0', grad_fn=<SelectBackward>)

In [55]:
rw_sentence_pairs[1]
# 0: entailment
# 1: neutral
# 2: contradiction

('hollywood actor morgan freeman said would like make movie india morgan india recently shoot documentary story god added loves india offer instance places gaya varanasi fascinating differences said',
 'new zealand unveils new bike with')

In [56]:
maxi = torch.max(prediction, dim=0)
mini = torch.min(prediction, dim=0)
# print(maxi[0][0])
denom1 = float(maxi[0][0].to('cpu')) - float(mini[0][0].to('cpu'))
denom2 = float(maxi[0][1].to('cpu')) - float(mini[0][1].to('cpu'))
denom3 = float(maxi[0][2].to('cpu')) - float(mini[0][2].to('cpu'))
denom = ( denom1 + denom2 + denom3 ) / 3
print(maxi, mini)
print(denom)

torch.return_types.max(
values=tensor([1.2348, 1.8643, 4.4961], device='cuda:0', grad_fn=<MaxBackward0>),
indices=tensor([46, 45, 36], device='cuda:0')) torch.return_types.min(
values=tensor([-3.0378, -0.9278, -1.3528], device='cuda:0', grad_fn=<MinBackward0>),
indices=tensor([36, 78, 46], device='cuda:0'))
4.304547866185506


In [57]:
aa = []
for i in prediction:
    a = []
    for j in i:
        a.append(float(j.to('cpu'))/denom)
    aa.append(a)
print(aa)

[[0.07257015702772558, 0.18065164509390388, -0.14787133818493023], [-0.21517429361007837, 0.13840414327317496, 0.1457022212661912], [0.22069163254159652, 0.2244886602841565, -0.276174320066889], [-0.02615162615134437, 0.26272665367852044, -0.11880807932287256], [0.22481402089851724, -0.046980943498979126, -0.20636092163269282], [0.09202962823085338, 0.04298122829279149, -0.10764087277787485], [0.23552743682965657, -0.00955954681274174, -0.22953046896847815], [0.04250791354962943, 0.02189098688094825, -0.049148433406942846], [0.15078335514712451, 0.03908698146381934, -0.1633602038278482], [0.14775273948458553, -0.0008792608550512579, -0.14536999258932365], [0.05938374702917064, 0.19509227269671725, -0.1445440942259774], [0.12475798157021892, 0.05634856188956056, -0.14475357013243564], [0.014667765230496333, 0.08879361457396956, -0.06897466399435032], [0.1413706062470161, -0.04284167577069585, -0.1255334910636814], [-0.06068186623056193, -0.14103533125210116, 0.18864120958987568], [0.133

In [58]:
e_scores = []
for i in aa:
    e_scores.append((i[0] + i[1]*0.5 - i[2])/2.5)
print(e_scores)

[0.1243069271038431, -0.11666977729587283, 0.24364411310022552, 0.08960791200431537, 0.1630737883126882, 0.08846444606204959, 0.18411125295670555, 0.04104073615881856, 0.13347481988275295, 0.11707324065855343, 0.12058959104140268, 0.11907433305897394, 0.05121569460473256, 0.09819330377013982, -0.1279362965785953, 0.09182109183657475, -0.1382031832103492, 0.0007350647749683952, 0.22359891940267723, 0.14905170888565006, 0.14644242393348456, 0.09682997051514919, 0.06102929672683096, 0.10216855288833693, 0.033677443419857934, 0.17347422621998437, -0.03333331887101533, 0.06227069694790503, 0.17076905148382737, 0.06415692723400397, 0.12447768292322907, 0.07647985630499309, 0.03485169932944263, 0.06191961017763412, 0.013955696002715618, 0.14645969932619324, -0.7359447731708813, 0.01269744652728697, 0.07183115488668095, 0.018183134560465324, 0.1502205506572505, 0.041654810181430024, 0.1133212589594291, 0.10830007384828824, 0.09183410446105215, 0.13100159595679378, 0.2650334032466402, 0.1077926

In [59]:
E_labels = []
for i in e_scores:
    if i < 0:
        E_labels.append('NEGATIVE ENTAILMENT')
    elif i > 0.3:
        E_labels.append('PERFECT ENTAILMENT')
    else:
        E_labels.append('NEUTRAL ENTAILMENT')

In [60]:
ctrr = 0
for i in e_scores:
    if i < 0:
        ctrr = ctrr + 1
print(ctrr)

9


In [62]:
ctrr = 0
for i in e_scores:
    if i > 0:
        ctrr = ctrr + 1
print(ctrr)

91


In [ ]:
df_final_E = pd.DataFrame(list(zip(rw_sentence_pairs, aa, e_scores, E_labels)), columns =['sentence pairs', 'entailment scores (raw: e, n, c)', 'entailment_metric [-1:1]', 'entailment_labels'])
df_final_E.to_csv('scores/summary_entailment_scores_of_test_pairs.csv')
df_final_E.head()

In [ ]:
maxi_1 = torch.max(prediction_1, dim=0)
mini_1 = torch.min(prediction_1, dim=0)
# print(maxi[0][0])
denom1_1 = float(maxi_1[0][0].to('cpu')) - float(mini_1[0][0].to('cpu'))
denom2_1 = float(maxi_1[0][1].to('cpu')) - float(mini_1[0][1].to('cpu'))
denom3_1 = float(maxi_1[0][2].to('cpu')) - float(mini_1[0][2].to('cpu'))
denom_1 = ( denom1_1 + denom2_1 + denom3_1 ) / 3
print(maxi_1, mini_1)
print(denom_1)

maxi_2 = torch.max(prediction_2, dim=0)
mini_2 = torch.min(prediction_2, dim=0)
# print(maxi[0][0])
denom1_2 = float(maxi_2[0][0].to('cpu')) - float(mini_2[0][0].to('cpu'))
denom2_2 = float(maxi_2[0][1].to('cpu')) - float(mini_2[0][1].to('cpu'))
denom3_2 = float(maxi_2[0][2].to('cpu')) - float(mini_2[0][2].to('cpu'))
denom_2 = ( denom1 + denom2 + denom3 ) / 3
print(maxi_2, mini_2)
print(denom_2)

In [ ]:
p1 = []
for i in prediction_1:
    p = []
    for j in i:
        p.append(float(j.to('cpu'))/denom_1)
    p1.append(p)
print(p1)

p2 = []
for i in prediction_2:
    p = []
    for j in i:
        p.append(float(j.to('cpu'))/denom_2)
    p2.append(p)
print(p2)

In [ ]:
me1_scores = []
for i in p1:
    me1_scores.append((i[0] + i[1]*0.5 - i[2])/2.5)
print(me1_scores)

me2_scores = []
for i in p2:
    me2_scores.append((i[0] + i[1]*0.5 - i[2])/2.5)
print(me2_scores)

In [ ]:
mutual_scores= []
for i, j in zip(me1_scores, me2_scores):
    mutual_scores.append( (i + j) / 2 )

In [ ]:
MUTUAL_labels = []
n_c = 0
p_c = 0
c_c = 0
for i in mutual_scores:
    if i <= -0.2:
        c_c += 1
        MUTUAL_labels.append('CONTRADICTION')
    elif i > -0.2 and i < 0:
        n_c += 1
        MUTUAL_labels.append('NEGATIVE ENTAILMENT')
    else:
        p_c += 1
        MUTUAL_labels.append('POSITIVE ENTAILMENT')
        
print("POSITIVE Entailments: ", p_c)
print("NEGATIVE Entailments: ", n_c)
print("Contradictions: ", c_c)

In [ ]:
df_final_ME = pd.DataFrame(list(zip(rw_sentence_pairs, p1, p2, me1_scores, me2_scores, mutual_scores, MUTUAL_labels)), columns =['sentence pairs', 'e_n_c scores 1', 'e_n_c scores 2', 'entailment_metric_1[-1:1]', 'entailment_metric_2[-1:1]', 'mutuality_score', 'mutual_labels'])
df_final_ME.to_csv('scores/mutual_entailment_scores_of_test_pairs.csv')
df_final_ME.head()